In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

%matplotlib inline
import tensorflow.python.keras.engine.base_layer_v1

In [2]:
df=pd.read_csv('BTC-Hourly.csv')

In [3]:
df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43312.27,52.056320,2.254677e+06
1,1646089200,2022-02-28 23:00:00,BTC/USD,43085.30,43364.81,42892.37,43178.98,106.816103,4.612210e+06
2,1646085600,2022-02-28 22:00:00,BTC/USD,41657.23,44256.08,41650.29,42907.32,527.540571,2.263535e+07
3,1646082000,2022-02-28 21:00:00,BTC/USD,41917.09,41917.09,41542.60,41659.53,69.751680,2.905822e+06
4,1646078400,2022-02-28 20:00:00,BTC/USD,41361.99,41971.00,41284.11,41914.97,247.151654,1.035935e+07


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33259 entries, 0 to 33258
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unix        33259 non-null  int64  
 1   date        33259 non-null  object 
 2   symbol      33259 non-null  object 
 3   open        33259 non-null  float64
 4   high        33259 non-null  float64
 5   low         33259 non-null  float64
 6   close       33259 non-null  float64
 7   Volume BTC  33259 non-null  float64
 8   Volume USD  33259 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 2.3+ MB


In [5]:
df.isnull().sum()

unix          0
date          0
symbol        0
open          0
high          0
low           0
close         0
Volume BTC    0
Volume USD    0
dtype: int64

In [6]:
df["date"]=pd.to_datetime(df["date"])
df["day"]=(df["date"]).dt.day
df["month"]=(df["date"]).dt.month
df["year"]=(df["date"]).dt.year
df["time"]=(df["date"]).dt.time
del df["date"]


In [7]:
df['time']=df['time'].astype(str) #We change the type of time to string.
df["time"]=df.time.str.replace(':00','') #We delete the minute part of time.
df['day']=df['day'].astype(int)
df['month']=df['month'].astype(int)
df['year']=df['year'].astype(int)
df['time']=df['time'].astype(int)

In [8]:
df.head()

,unix,symbol,open,high,low,close,Volume BTC,Volume USD,day,month,year,time
0,1646092800,BTC/USD,43221.71,43626.49,43185.48,43312.27,52.056320,2.254677e+06,1,3,2022,0
1,1646089200,BTC/USD,43085.30,43364.81,42892.37,43178.98,106.816103,4.612210e+06,28,2,2022,23
2,1646085600,BTC/USD,41657.23,44256.08,41650.29,42907.32,527.540571,2.263535e+07,28,2,2022,22
3,1646082000,BTC/USD,41917.09,41917.09,41542.60,41659.53,69.751680,2.905822e+06,28,2,2022,21
4,1646078400,BTC/USD,41361.99,41971.00,41284.11,41914.97,247.151654,1.035935e+07,28,2,2022,20


In [9]:
abs(df.drop('symbol',axis=1).corr()["high"].sort_values(ascending=False))

high          1.000000
open          0.999958
close         0.999958
low           0.999905
unix          0.824581
year          0.801393
Volume USD    0.482761
time          0.000503
day           0.012292
month         0.039328
Volume BTC    0.324982
Name: high, dtype: float64

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
import tensorflow as tf
import keras
from tensorflow import keras

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [13]:
x,y=df.drop(["low","close","open","high","symbol"],axis=1),df[["high"]]
x=scaler.fit_transform(x)
x.shape

(33259, 7)

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [15]:
model=Sequential()
model.add(Dense(84,activation="relu"))
model.add(Dense(84,activation="relu"))
model.add(Dense(84,activation="relu"))
model.add(Dense(84,activation="relu"))
model.add(Dense(84,activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam",loss="mse")

In [16]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=128,epochs=100)
model.summary()

Epoch 1/100
208/208 [==============================] - 3s 5ms/step - loss: 416577120.0000 - val_loss: 146949968.0000
Epoch 2/100
208/208 [==============================] - 1s 4ms/step - loss: 92244296.0000 - val_loss: 67721544.0000
Epoch 3/100
208/208 [==============================] - 1s 4ms/step - loss: 57846252.0000 - val_loss: 51398920.0000
Epoch 4/100
208/208 [==============================] - 1s 4ms/step - loss: 50016616.0000 - val_loss: 47327584.0000
Epoch 5/100
208/208 [==============================] - 1s 3ms/step - loss: 46612956.0000 - val_loss: 43948272.0000
Epoch 6/100
208/208 [==============================] - 1s 4ms/step - loss: 43129836.0000 - val_loss: 40441404.0000
Epoch 7/100
208/208 [==============================] - 1s 3ms/step - loss: 39799988.0000 - val_loss: 37956872.0000
Epoch 8/100
208/208 [==============================] - 1s 3ms/step - loss: 37032292.0000 - val_loss: 35365132.0000
Epoch 9/100
208/208 [==============================] - 1s 3ms/step - loss: 346

In [17]:
tahmin=model.predict(x_test)

208/208 [==============================] - 0s 1ms/step


In [18]:
r2_score(tahmin,y_test)

0.989933537492834

array([[ 7629.1855],
       [ 8697.338 ],
       [10297.362 ],
       ...,
       [10098.811 ],
       [ 9536.91  ],
       [10315.992 ]], dtype=float32)